<a href="https://colab.research.google.com/github/toshNaik/Image-Colorizer/blob/master/ImageColorizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import math
import glob
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from skimage import color, io, transform
from tensorflow.keras.models import Model

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [4]:
!tar -xf /content/drive/My\ Drive/datasets/color-net.tar

In [106]:
image_path_list = glob.glob('Train/*')
data = tf.data.Dataset.from_tensor_slices(image_path_list)

In [ ]:
len(image_path_list)

In [6]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

def tf_rgb2lab(image):
  im_shape = image.shape
  [image,] = tf.py_function(color.rgb2lab, [image], [tf.float32])
  image.set_shape(im_shape)
  return image

def preprocess(path):
  image = tf.io.read_file(path)
  image = tf.image.decode_jpeg(image, channels=3)
  image = tf.image.convert_image_dtype(image, tf.float32)
  image = tf.image.resize(image, [224, 224])
  image = tf_rgb2lab(image)
  L = image[:,:,0]/100.
  ab = image[:,:,1:]/128.
  input = tf.stack([L,L,L], axis=2)
  return input, ab

train_ds = data.shuffle(10000, reshuffle_each_iteration=True).map(preprocess, AUTOTUNE).repeat().batch(32).prefetch(AUTOTUNE)

In [10]:
vggmodel = tf.keras.applications.VGG16(include_top=False, weights='imagenet', input_shape=(224,224,3))
model = tf.keras.Sequential()
for i,layer in enumerate(vggmodel.layers):
  model.add(layer)
for layer in model.layers:
  layer.trainable=False

In [11]:
model.add(tf.keras.layers.Conv2D(256, (3,3), padding='same', activation='relu'))
model.add(tf.keras.layers.UpSampling2D((2,2)))
model.add(tf.keras.layers.Conv2D(128, (3,3), padding='same', activation='relu'))
model.add(tf.keras.layers.UpSampling2D((2,2)))
model.add(tf.keras.layers.Conv2D(64, (3,3), padding='same', activation='relu'))
model.add(tf.keras.layers.UpSampling2D((2,2)))
model.add(tf.keras.layers.Conv2D(16, (3,3), padding='same', activation='relu'))
model.add(tf.keras.layers.UpSampling2D((2,2)))
model.add(tf.keras.layers.Conv2D(8, (3,3), padding='same', activation='relu'))
model.add(tf.keras.layers.Conv2D(2, (3,3), padding='same', activation='tanh'))
model.add(tf.keras.layers.UpSampling2D((2,2)))

In [12]:
model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
model_checkpoint_callback = ModelCheckpoint('/content/drive/My Drive/models/colorizer/color-net_weights.{epoch:02d}.hdf5',
                                            monitor='loss',
                                            mode='min')
early_stopping_callback = EarlyStopping(monitor='loss', mode='min', verbose=1, patience=5)
steps = len(image_path_list) // 32
history = model.fit(train_ds,
                    epochs=500,
                    steps_per_epoch=steps,
                    callbacks=[model_checkpoint_callback])

Epoch 1/500
290/290 [==============================] - 126s 436ms/step - loss: 0.0101 - accuracy: 0.5946
Epoch 2/500
290/290 [==============================] - 126s 435ms/step - loss: 0.0098 - accuracy: 0.6086
Epoch 3/500
290/290 [==============================] - 126s 434ms/step - loss: 0.0097 - accuracy: 0.6088
Epoch 4/500
290/290 [==============================] - 120s 413ms/step - loss: 0.0096 - accuracy: 0.6119
Epoch 5/500
290/290 [==============================] - 118s 407ms/step - loss: 0.0096 - accuracy: 0.6139
Epoch 6/500
290/290 [==============================] - 117s 402ms/step - loss: 0.0095 - accuracy: 0.6167
Epoch 7/500
290/290 [==============================] - 117s 405ms/step - loss: 0.0094 - accuracy: 0.6180
Epoch 8/500
290/290 [==============================] - 116s 401ms/step - loss: 0.0094 - accuracy: 0.6177
Epoch 9/500
290/290 [==============================] - 111s 384ms/step - loss: 0.0094 - accuracy: 0.6198
Epoch 10/500
290/290 [==============================] -

In [12]:
from tensorflow.keras.models import load_model
model = load_model('/content/drive/My Drive/models/colorizer/color-net_weights.328.hdf5')

In [ ]:
test_images = glob.glob('Test/*')
len(test_images)

In [ ]:
image = io.imread(test_images[256])
image = transform.resize(image, [224, 224])
plt.imshow(image)

In [ ]:
image = color.rgb2gray(image)
print(np.amax(image))
plt.imshow(image, cmap='gray')

In [ ]:
L = np.stack((image,image,image), axis=2)
input = np.expand_dims(L, axis=0)
input.shape

In [126]:
ab = model.predict(input)

In [ ]:
ab = np.squeeze(ab, axis=0)
print(ab.shape)

In [ ]:
image = np.expand_dims(image, axis=-1)
image.shape

In [ ]:
ori = np.concatenate((image*100, ab*128), axis=-1)
print(ori.shape)
rgb = color.lab2rgb(ori)

In [ ]:
plt.imshow(rgb)